In [192]:
import requests
import json
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tn
from lxml import html
import pandas as pd

In [193]:
rbi_link='https://www.rbi.org.in/Scripts/AnnualPublications.aspx?head=Handbook%20of%20Statistics%20on%20Indian%20Economy'
headers=requests.utils.default_headers()

- handling 104 error: https://stackoverflow.com/questions/43165341/python3-requests-connectionerror-connection-aborted-oserror104-econnr

In [194]:
headers['User-Agent']='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'

In [195]:
rbi=requests.get(rbi_link,headers=headers) # but I don;t it works without without headers specifically given into request.get()
print(rbi.status_code)

200


In [196]:
soup=bs(rbi.text,features='html.parser')
# print(soup.prettify())

In [197]:
## find all available links/PDF links on rbi handbook website
rbi_all_pdf_link=[]
rbi_links=[]
for link in soup.find_all('a'):
    s=link.get('href')
    rbi_links.append(s)
    if s!=None: # if link is not none condition on PDF
        if s.find('PDF')!=-1: # doing special finding implementation about this webpage
            rbi_all_pdf_link.append(link.get('href'))

In [198]:
## to get all the text on webpage
all_text=soup.get_text()

In [199]:
link_title=[]
link_url=[]
link_size=[]
total_size=0 ## possible with bit string work
for link in soup.find(attrs={'class':'tablebg'}).find_all('tr'):
    s=link.find('a')
    if s!=None:
        pdf_s=s.get('target')
        if pdf_s=='_blank':
            link_url.append(s.get('href'))
            text=link.get_text().split('\xa0')
            link_title.append(text[0])
            link_size.append(text[-1])
            total_size+=float(text[-1].split(' ')[1])
    else:
        link_url.append('table header: no link')
        text=link.get_text().split('\xa0')
        link_title.append(text[0])
        link_size.append(' 0 kb') # as no link so no pdf to get 
        total_size+=0

In [200]:
print('Total size of the pdfs:',round(total_size/1000,1),'MB')

Total size of the pdfs: 38.1 MB


In [201]:
rbi_handbook_df=pd.DataFrame({'pdfName':link_title,'pdfLink':link_url,'pdfSize':link_size})

# making links clickable
def make_clickable(val):
    # target _blank to open new window
    if val!='table header: no link':
        return '<a target="_blank" href="{}">{}</a>'.format(val, val) # target blank given to open link in new tab
    else:
        return val

rbi_handbook_df=rbi_handbook_df.style.format({'pdfLink': make_clickable})
### uncomment to save as csv
#rbi_handbook_df.to_csv('rbi-handbook-website-pdfs.csv',index=False)